# Nilearn Tutorial 5
With DS109 dataset taken from openfmri.org --- False-belief experiment

In [1]:
%matplotlib inline 
import seaborn
import nibabel as nib
import nilearn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fb_sub2run1 = nib.load('/home/farahana/Documents/dataset/Multi Subject/ds109/sub002/BOLD/task001_run001/bold.nii')
print (fb_sub2run1.shape)

fb_sub2run1_label = np.genfromtxt('/home/farahana/Documents/dataset/Multi Subject/ds109/sub002/behav/task001_run001/behavdata.txt', skip_header=1)
print (fb_sub2run1_label.shape)

(72, 72, 36, 179)
(179, 6)


In [3]:
# take StoryCode in behavdata.txt as labels or targets for fMRI dataset
fb_sub2run1_labels = fb_sub2run1_label[:,2]

In [4]:
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(memory='nilearn_cache', mask_strategy='epi')
masker.fit(fb_sub2run1)
fb_s2r1_fmri = masker.transform(fb_sub2run1)
print (fb_s2r1_fmri.shape)

(179, 37551)


## Data Preparation

In [5]:
### Cross validation
from sklearn.cross_validation import cross_val_score, KFold
cv = KFold(n=len(fb_s2r1_fmri), n_folds=10)

In [6]:
### Train_test Splitting
from sklearn.cross_validation import train_test_split
fb_s2r1_fmri_train, fb_s2r1_fmri_test, fb_s2r1_label_train, fb_s2r1_label_test = train_test_split(fb_s2r1_fmri,fb_sub2run1_labels, random_state = 30)

## 1. SVC

In [7]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear')

In [9]:
svc1 = svc_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_svc = cross_val_score(svc1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_svc))

0.430065359477


In [10]:
svc2 = svc_classifier.fit(fb_s2r1_fmri_train, fb_s2r1_label_train)
ts_score_svc = svc2.score(fb_s2r1_fmri_test, fb_s2r1_label_test)
print(ts_score_svc)

0.555555555556


## 2. Naive Bayes

In [11]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb_classifier = GaussianNB()

In [12]:
gnb1 = gnb_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_gnb = cross_val_score(gnb1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_gnb))

0.314052287582


In [13]:
gnb2 = gnb_classifier.fit(fb_s2r1_fmri_train, fb_s2r1_label_train)
ts_score_gnb = gnb2.score(fb_s2r1_fmri_test, fb_s2r1_label_test)
print(ts_score_gnb)

0.466666666667


In [14]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
mnb_classifier = MultinomialNB()

In [15]:
mnb1 = mnb_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_mnb = cross_val_score(mnb1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_mnb))

0.0611111111111


In [16]:
mnb2 = mnb_classifier.fit(fb_s2r1_fmri_train, fb_s2r1_label_train)
ts_score_mnb = mnb2.score(fb_s2r1_fmri_test, fb_s2r1_label_test)
print(ts_score_mnb)

0.377777777778


## 3. KNN

In [17]:
# K-nearest neighbors (k=5)
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()

In [18]:
knn1 = knn_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_knn = cross_val_score(knn1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_knn))

0.340522875817


In [19]:
knn2 = knn_classifier.fit(fb_s2r1_fmri_train, fb_s2r1_label_train)
ts_score_knn = knn2.score(fb_s2r1_fmri_test, fb_s2r1_label_test)
print(ts_score_knn)

0.644444444444


## 4. Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

In [21]:
dt1 = dt_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_dt = cross_val_score(dt1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_dt))

0.307189542484


In [22]:
dt2 = dt_classifier.fit(fb_s2r1_fmri_train, fb_s2r1_label_train)
ts_score_dt = dt2.score(fb_s2r1_fmri_test, fb_s2r1_label_test)
print(ts_score_dt)

0.222222222222


## 5. Esemble Method

In [23]:
# Bagging meta-estimator
from sklearn.ensemble import BaggingClassifier
bag_classifier = BaggingClassifier(knn_classifier)

In [ ]:
dt1 = dt_classifier.fit(fb_s2r1_fmri, fb_sub2run1_labels)
cv_score_dt = cross_val_score(dt1, fb_s2r1_fmri, fb_sub2run1_labels, cv=cv)
print (np.mean(cv_score_dt))